<a href="https://colab.research.google.com/github/DartDoesData/build-within-ai-overview/blob/main/Using_Nominatim_and_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reading from a Database and Preparing for Address Correction**

In this section, you'll learn how to connect to an SQLite database, query data into a pandas DataFrame, and prepare for address correction using Nominatim (an open-source geocoding tool). This is a foundational skill in working with AI tools that often require structured data.

---

## **What We'll Do**
1. Download an SQLite database file from GitHub.
2. Connect to the database using Python.
3. Query data and load it into a pandas DataFrame for easy manipulation.
4. Prepare the data for address correction with Nominatim.

### **Step-by-Step Explanation**
1. **Import Libraries**:
   - `sqlite3`: For connecting to SQLite databases.
   - `pandas`: For working with data in table-like structures.
   - `requests`: To download the database file from a URL.

2. **Download the Database File**:
   - The database is hosted on GitHub, and we use `requests` to fetch it.
   - The file is saved locally as `students-demo.db`.
   - Note that for this is for demonstration purposes. Best practice is to leverage a hosted database for live interactions.

3. **Connect to the Database**:
   - Use `sqlite3.connect` to establish a connection to the database.

4. **Query the Data**:
   - Write an SQL query (`SELECT * FROM students`) to retrieve all records from the `students` table.
   - Load the query results into a pandas DataFrame for further processing.

5. **Close the Connection**:
   - Always close the database connection after querying to free up resources.

6. **Display the Data**:
   - Use `students_df.head()` to preview the first few rows of the data.

In [ ]:
# Step 1: Import libraries
import sqlite3
import pandas as pd
import requests

db_url = "https://raw.githubusercontent.com/DartDoesData/build-within-ai-overview/main/db/students-demo.db"
db_file = "students-demo.db"

try:
    # Step 2: Download the SQLite DB file from GitHub
    response = requests.get(db_url)
    response.raise_for_status()
    with open(db_file, "wb") as f:
        f.write(response.content)
except requests.exceptions.RequestException as e:
    print(f"Error downloading database: {e}")
    exit()

try:
    # Step 3: Connect to the SQLite Database
    conn = sqlite3.connect(db_file)

    # Step 4: Query the database and load into Pandas DataFrame
    query = "SELECT * FROM students"
    students_df = pd.read_sql_query(query, conn)

finally:
    # Step 5: Close the connection
    if conn:
        conn.close()

# Step 6: Display the DataFrame
display(students_df)

,id_student,student_id,last_name,first_name,full_address,street_number,street_name,city,state,zip,lat,lng,display_name,address_narrative,name_narrative
0,1,1001,Williams,Dartanion,6040 Harper Chicago IL,None,None,None,None,None,None,None,None,None,None
1,2,1002,Swift,Deangelo,250 K St NE DC,None,None,None,None,None,None,None,None,None,None
2,3,1003,Swift,Aramis,"1900 Connecticut Ave NW, DC",None,None,None,None,None,None,None,None,None,None


Here's the revised code and markdown, rewritten to align perfectly with redefined steps.

---

### **Markdown**

# **Validating and Enriching Addresses with Nominatim**

Now that we have the data, our next task is to correct and validate addresses using **Nominatim**, a free geocoding tool. This tool converts address information into detailed components such as latitude, longitude, city, state, and postal code.

In this section, you'll learn how to use Nominatim to validate and enrich addresses. This builds on the data we loaded from the SQLite database and prepares it for use in AI-powered applications.

---

## **What We'll Do**
1. Install and import necessary libraries.
2. Set up the Nominatim geolocator.
3. Define a function to process addresses and retrieve detailed components.
4. Handle errors gracefully to ensure consistent output.
5. Apply the function to the dataset and display the updated DataFrame.

---

### **Step-by-Step Explanation**
1. **Install and Import Libraries**:
   - Install the `geopy` library, which provides geocoding capabilities.
   - Import `geopy` tools, `pandas`, and any necessary error-handling modules.

2. **Set Up the Nominatim Geolocator**:
   - Create a Nominatim geolocator instance with a unique user agent string.
   - This step prepares the geolocator to process address inputs.

3. **Define the Address Parsing Function**:
   - Use `geolocator.geocode` to find geographic coordinates (latitude, longitude) for a given address.
   - Use `geolocator.reverse` to translate coordinates into detailed address components.
   - If no data is found, return a default template with empty values.

4. **Apply the Function to the Dataset**:
   - Use the `parse_address` function to process each address in the dataset.
   - Expand the parsed details into new columns in the DataFrame.

5. **Preview the Updated DataFrame**:
   - Display the enriched data, including both the original address and the newly added details.

In [ ]:
# Step 1: Install the geopy library for geocoding capabilities
!pip install geopy --quiet

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Step 2: Set Up the Nominatim Geolocator
geolocator = Nominatim(user_agent="build_within")

# Step 3: Define the Address Parsing Function
# Return a default template with empty values if parsing fails
EMPTY_RESULT = {
    'street_number': None,
    'street_name': None,
    'city': None,
    'state': None,
    'zip': None,
    'lat': None,
    'lng': None,
    'display_name': None
}

def parse_address(address):
    try:
        # Retrieve geographic coordinates for the address
        location = geolocator.geocode(address, timeout=10)
        if location:
            # Retrieve detailed address components based on the coordinates
            details = geolocator.reverse(
                (location.latitude, location.longitude),
                timeout=10,
                addressdetails=True
            ).raw.get('address', {})
            return {
                'street_number': details.get('house_number'),
                'street_name': details.get('road'),
                'city': details.get('city', details.get('town', details.get('village'))),
                'state': details.get('state', details.get('region')),
                'zip': details.get('postcode'),
                'lat': location.latitude,
                'lng': location.longitude,
                'display_name': location.address
            }
    except GeocoderTimedOut:
        # Handle timeout errors and return default empty result
        pass

    # Return the default empty result for failures
    return EMPTY_RESULT

# Step 5: Apply the Function to the Dataset
# Apply the parse_address function to each row in the DataFrame
# Expand the parsed details into separate columns
students_df[['street_number', 'street_name', 'city', 'state', 'zip', 'lat', 'lng', 'display_name']] = \
    students_df['full_address'].apply(lambda x: pd.Series(parse_address(x)))

# Step 6: Preview the Updated DataFrame
display(students_df.head())


,id_student,student_id,last_name,first_name,full_address,street_number,street_name,city,state,zip,lat,lng,display_name,address_narrative,name_narrative
0,1,1001,Williams,Dartanion,6040 Harper Chicago IL,6040,South Harper Avenue,Chicago,Illinois,60637,41.784468,-87.588761,"6040, South Harper Avenue, Woodlawn, Chicago, ...",None,None
1,2,1002,Swift,Deangelo,250 K St NE DC,250,K Street Northeast,Washington,None,20426,38.903209,-77.002329,"Loree Grand, 250, K Street Northeast, NoMa, Ne...",None,None
2,3,1003,Swift,Aramis,"1900 Connecticut Ave NW, DC",1900,Connecticut Avenue Northwest,Washington,None,20036,38.916028,-77.046782,"1900, Connecticut Avenue Northwest, Ward 2, Wa...",None,None


# **Generating AI-Powered Narratives with OpenAI**

In this step, we will use OpenAI's GPT model to generate narratives for each record in our dataset. This process demonstrates how to integrate generative AI into a real-world data enrichment workflow.

---

## **What We'll Do**
1. Import necessary libraries and retrieve the API key.
2. Define key parameters for interacting with the OpenAI API.
3. Write a function to generate AI responses based on custom prompts.
4. Iterate through the dataset and update it with AI-generated narratives.
5. Display the dataset.

---

### **How the code works**

1. **Import Necessary Libraries**:
   - Import `os`, `userdata`, and `requests` for API interaction and key management.

2. **Retrieve the OpenAI API Key**:
   - Use `userdata.get` to retrieve the API key from Colab Secrets.
   - If the key is missing, raise an exception with instructions for adding it.

3. **Define Constants and Parameters**:
   - Set the maximum token limit for the LLM response (`MAX_TOKENS`).
   - Configure verbosity, mood, and tone parameters for the assistant's behavior.
   - Define the API endpoint and required headers.

4. **Write the Function to Generate AI Responses**:
   - Send a prompt to the OpenAI API via a POST request.
   - Parse the JSON response to extract the generated content.
   - Handle errors gracefully by raising descriptive exceptions.

5. **Iterate Through the Dataset**:
   - Loop through each row of the `students_df` DataFrame.
   - For each row:
     - Generate a narrative about the city and state using the `generate_response` function.
     - Generate a narrative about the first name using a separate prompt.
   - Update the DataFrame with the generated narratives.

6. **Display the Updated Dataset**:
   - Show the dataset with new columns for `address_narrative` and `name_narrative`.

In [ ]:
# Step 1: Import necessary libraries
import os
from google.colab import userdata
import requests

# Step 2: Retrieve the OpenAI API key
# Attempt to retrieve the API key from Colab Secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

if OPENAI_API_KEY:
    print('API key retrieved from Colab Secrets.')
else:
    raise Exception('API key not found in Colab Secrets. Please add it under "Secrets".')

# Step 3: Define constants and parameters
# Maximum tokens for the LLM response
MAX_TOKENS = 1024

# Configurable parameters for LLM behavior
verbosity = 'brief'
mood = 'optimistic'
tone = 'direct'

# OpenAI API endpoint and headers
openai_endpoint = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
    'Content-Type': 'application/json'
}

# Step 4: Function to generate LLM response
# This function sends a prompt to the OpenAI API and returns the response content
def generate_response(prompt):
    request_payload = {
        'model': 'gpt-4o',
        'messages': [
            {'role': 'system', 'content': f'You are an assistant with {verbosity} verbosity. Your mood is {mood}, and you speak with a {tone} tone.'},
            {'role': 'user', 'content': prompt}
        ],
        'max_tokens': MAX_TOKENS
    }

    response = requests.post(openai_endpoint, headers=headers, json=request_payload)
    if response.status_code == 200:
        response_json = response.json()
        return response_json['choices'][0]['message']['content'].strip()
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

# Step 5: Iterate through the DataFrame and update narratives
# Loop through each row in the DataFrame to generate AI-generated narratives
for idx, row in students_df.iterrows():
    try:
        # Generate narrative for city and state
        address_prompt = f"What are three popular things to do in {row['city']}, {row['state']}?"
        address_fact = generate_response(address_prompt)
        students_df.loc[idx, 'address_narrative'] = address_fact

        # Generate narrative for first name
        name_prompt = f"Tell a brief, fun fact about the name {row['first_name']}."
        name_fact = generate_response(name_prompt)
        students_df.loc[idx, 'name_narrative'] = name_fact

    except Exception as e:
        # Log any errors during the iteration
        print(f"Error processing row {idx}: {e}")

# Step 6: Display the updated DataFrame
# Show the DataFrame with new AI-generated narratives
display(students_df)

API key retrieved from Colab Secrets.


,id_student,student_id,last_name,first_name,full_address,street_number,street_name,city,state,zip,lat,lng,display_name,address_narrative,name_narrative
0,1,1001,Williams,Dartanion,6040 Harper Chicago IL,6040,South Harper Avenue,Chicago,Illinois,60637,41.784468,-87.588761,"6040, South Harper Avenue, Woodlawn, Chicago, ...",1. Visit Millennium Park.\n2. Explore the Art ...,The name Dartanion is a unique variation of D'...
1,2,1002,Swift,Deangelo,250 K St NE DC,250,K Street Northeast,Washington,None,20426,38.903209,-77.002329,"Loree Grand, 250, K Street Northeast, NoMa, Ne...","I assume you meant Washington, D.C. Here are t...","Deangelo is a name that combines ""De"" with ""An..."
2,3,1003,Swift,Aramis,"1900 Connecticut Ave NW, DC",1900,Connecticut Avenue Northwest,Washington,None,20036,38.916028,-77.046782,"1900, Connecticut Avenue Northwest, Ward 2, Wa...","It seems you meant Washington, D.C. Here are t...","The name Aramis is famously known from ""The Th..."


# **Working with LLM-Generated Code in a Parsed Format**

When working with code or responses generated by a Language Model (LLM), it's often helpful to parse the responses into structured data. This approach ensures consistency and usability, particularly when dealing with hierarchical or tabular information like recipes.

---

## **What We'll Do**
1. Define dataclasses to organize structured data.
2. Create functions to parse JSON responses into these dataclasses.
3. Configure an OpenAI API request to retrieve structured recipe data.
4. Parse the API response into a structured format.
5. Convert the structured data into pandas DataFrames for further analysis.
6. Display the results for easy interpretation.

---

### **How the code works**

1. **Define Dataclasses**:
   - `Ingredient`, `Step`, `CookingMethod`, and `Recipe` dataclasses represent the hierarchy of recipe components.
   - These classes make it easier to parse and manipulate structured JSON data.

2. **Create Parsing Functions**:
   - `parse_ingredients`: Converts a list of ingredient dictionaries into `Ingredient` objects.
   - `parse_steps`: Converts a list of step dictionaries into `Step` objects.
   - `parse_cooking_modes`: Converts a list of cooking method dictionaries into `CookingMethod` objects, each containing its steps.

3. **Configure the OpenAI API Request**:
   - Define the API endpoint, headers, and request payload.
   - Request the recipe in a structured JSON format.

4. **Send the API Request and Handle the Response**:
   - Use `requests.post` to send the payload to the OpenAI API.
   - Parse the response into JSON, handling edge cases like extra text outside the JSON structure.

5. **Parse the Response into Structured Data**:
   - Use the dataclasses and parsing functions to organize the response into `Recipe`, `Ingredient`, and `CookingMethod` objects.

6. **Convert Structured Data into DataFrames**:
   - Create a pandas DataFrame for ingredients, showing the name, quantity, and unit.
   - Create another DataFrame for cooking methods, including the method, step number, and step description.

7. **Display the Results**:
   - Print and display the DataFrames to review the structured recipe details.

In [ ]:
import json
from dataclasses import dataclass
from typing import List
import requests
import pandas as pd

# Step 1: Define dataclasses for structured responses
# These classes represent the recipe structure for clear parsing and organization
@dataclass
class Ingredient:
    name: str
    quantity: str
    unit: str

@dataclass
class Step:
    step: int
    description: str

@dataclass
class CookingMethod:
    method: str  # e.g., "microwave", "bake", "fry", etc.
    steps: List[Step]  # List of steps specific to this method

@dataclass
class Recipe:
    title: str
    ingredients: List[Ingredient]  # List of Ingredient objects
    cooking_modes: List[CookingMethod]  # List of CookingMethod objects

# Step 2: Functions to parse JSON response
# These functions convert raw JSON into structured dataclass objects
def parse_ingredients(ingredient_list):
    return [Ingredient(**item) for item in ingredient_list]

def parse_steps(steps_list):
    return [Step(**step) for step in steps_list]

def parse_cooking_modes(modes_list):
    methods = []
    for mode in modes_list:
        method = mode['method']
        steps = parse_steps(mode['steps'])
        methods.append(CookingMethod(method=method, steps=steps))
    return methods

# Step 3: Prepare the OpenAI API request
# Define the API endpoint, headers, and request data
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
    'Content-Type': 'application/json'
}

meal = input("Enter the name of the dish: ")

MAX_TOKENS = 1024
data = {
    'model': 'gpt-4o',
    'messages': [
        {
            'role': 'system',
            'content': 'You are a chef assistant. Respond only with valid JSON. Do not include additional commentary, explanations, or extra text.'
        },
        {
            'role': 'user',
            'content': f"""
            Provide a recipe for {meal} in the following JSON format:
            {{
                "title": "<Recipe Title>",
                "ingredients": [
                    {{"name": "<Ingredient Name>", "quantity": "<Quantity>", "unit": "<Unit>"}}
                ],
                "cooking_modes": [
                    {{
                        "method": "<Cooking Method>",
                        "steps": [
                            {{"step": <Step Number>, "description": "<Step Description>"}}
                        ]
                    }}
                ]
            }}
            """
        }
    ],
    'max_tokens': MAX_TOKENS
}

# Step 4: Send the request and handle the response
response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    try:
        recipe_text = response.json()['choices'][0]['message']['content'].strip()

        # Clean and parse the JSON response
        if recipe_text.startswith("{") and recipe_text.endswith("}"):
            recipe_data = json.loads(recipe_text)
        else:
            json_start = recipe_text.find("{")
            json_end = recipe_text.rfind("}") + 1
            recipe_data = json.loads(recipe_text[json_start:json_end])

        # Step 5: Parse structured data
        ingredients = parse_ingredients(recipe_data['ingredients'])
        cooking_modes = parse_cooking_modes(recipe_data['cooking_modes'])
        recipe = Recipe(title=recipe_data['title'], ingredients=ingredients, cooking_modes=cooking_modes)

        # Step 6: Convert structured data to DataFrames
        ingredients_df = pd.DataFrame([{
            'Name': ing.name, 'Quantity': ing.quantity, 'Unit': ing.unit
        } for ing in recipe.ingredients])

        cooking_modes_data = []
        for mode in recipe.cooking_modes:
            for step in mode.steps:
                cooking_modes_data.append({
                    'Method': mode.method,
                    'Step': step.step,
                    'Description': step.description
                })
        cooking_modes_df = pd.DataFrame(cooking_modes_data)

        # Step 7: Display the results
        print("\nIngredients DataFrame:")
        display(ingredients_df)

        print("\nCooking Modes DataFrame:")
        display(cooking_modes_df)

    except json.JSONDecodeError as e:
        print("Error: The response could not be parsed as JSON. Please check the output format.")
        print("Parsing Error Details:", e)
else:
    print(f"Error: {response.status_code} - {response.text}")


Enter the name of the dish: ravioli

Ingredients DataFrame:


,Name,Quantity,Unit
0,All-purpose flour,2,cups
1,Eggs,3,large
2,Salt,0.5,teaspoon
3,Spinach,1,cup
4,Ricotta cheese,1,cup
5,Parmesan cheese,0.5,cup
6,Nutmeg,0.25,teaspoon
7,Black pepper,0.25,teaspoon
8,Water,2,tablespoons
9,Olive oil,1,tablespoon



Cooking Modes DataFrame:


,Method,Step,Description
0,Pasta Making,1,Mix flour and salt in a bowl.
1,Pasta Making,2,Make a well in the center and add eggs and water.
2,Pasta Making,3,Mix to form a dough.
3,Pasta Making,4,Knead dough for 8-10 minutes until smooth.
4,Pasta Making,5,Wrap dough in plastic wrap and rest for 30 min...
5,Filling Preparation,1,Cook spinach and squeeze out excess moisture.
6,Filling Preparation,2,"Mix spinach, ricotta, Parmesan, nutmeg, and bl..."
7,Filling Preparation,3,Chill the filling until ready to use.
8,Ravioli Assembly,1,Roll out pasta dough into thin sheets.
9,Ravioli Assembly,2,Place small amounts of filling onto one sheet.
